In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose


R provides numerous datasets that are accessible through Python

In [2]:
from statsmodels.datasets import get_rdataset

# Access R datasets
ausbeer = get_rdataset('ausbeer', 'fpp2').data


In [ ]:
df = pd.read_csv("AUS monthly beer production.csv")
df

# Seasonal decomposition

In [ ]:
plt.figure(figsize=(26,14))
# set index to "time"


# call decomposition model (seasonal_decompose)


# autocorrelation raw data

The data are quaterly (4 observations per year). So for a good analysis:
16 (4quarters * 4 years) or 20 lags (4quarters * 5 years)  
Important is to not exceed: 218/4

If nlags is not specified in the acf --> the standard n=24

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# calculate autocorrelation


In [ ]:
# plot acf, changing the number of lags


In [ ]:
# plot pacf


# correlation stationary data

## check stationarity

In [6]:
from statsmodels.tsa.stattools import adfuller, kpss

# test to check if the data are stationary
def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f'{name}:')
    print(f'  ADF Statistic: {result[0]:.4f}')
    print(f'  p-value: {result[1]:.6f}')
    print(f'  Stationary: {result[1] < 0.05}\n')

In [ ]:
# Original series
y_beer = df['value']

# Step 1: Quaterly differencing (lag=4 for quarterly data)
y_seasonal_diff = y_beer.diff(4).dropna()

# Step 2: First differencing (if needed after seasonal diff)
y_stationary_beer = y_seasonal_diff.diff(1).dropna()

# Check stationarity at each step
check_stationarity(y_beer, 'Original')
check_stationarity(y_seasonal_diff, 'After seasonal diff (lag=4)')
check_stationarity(y_stationary_beer, 'After seasonal + first diff')

Original:
  ADF Statistic: -2.6113
  p-value: 0.090677
  Stationary: False

After seasonal diff (lag=4):
  ADF Statistic: -2.0197
  p-value: 0.278018
  Stationary: False

After seasonal + first diff:
  ADF Statistic: -7.3529
  p-value: 0.000000
  Stationary: True



## acf and pacf

In [ ]:
# plot acf

In [ ]:
# plot pacf
